In [1]:
##https://colab.research.google.com/github/trekhleb/machine-learning-experiments/blob/master/experiments/sketch_recognition_cnn/sketch_recognition_cnn.ipynb#scrollTo=ZTO-30-IMfK3
##how to feed grayscale images to pretrained models which requires 3 input channels:
##https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images

from PIL import Image
import numpy as np
import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import math
import datetime
import platform
import pathlib
import random
import pandas as pd
# mogrify

In [2]:
# !pip install tensorflow==2.7.0
# !pip install -q tf-models-official==2.7.0
# !pip install -U opencv-python

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0:2], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 2 Logical GPU


In [5]:
sketch = pd.read_csv('data/sketch_cleaned.csv',encoding = 'unicode_escape')

len(sketch)

1102

In [6]:
from os.path import exists
import statistics

image = {}
lst = []
intensity_list = []
median_list = []
for i in range(0,1127):
# for i in [11,10]:
    if exists('data/Frothers-reduced-gray/Page-'+str(i)+'.png') and 'Page-'+str(i) in list(sketch.Image_ID):
        lst = lst + ['Page-'+str(i)]
        img = cv2.imread('data/Frothers-reduced-gray/Page-'+str(i)+'.png')

        A = cv2.resize(img, (int(0.3*img.shape[1]),int(0.3*img.shape[0])), interpolation = cv2.INTER_AREA)
#         A = cv2.resize(img, (int(0.4*img.shape[1]),int(0.4*img.shape[0])), interpolation = cv2.INTER_AREA) ##(525,1030)
        B = np.ones((309,309,3))*255 
#         B = np.ones((476,476,3))*255 
        B[0:A.shape[0],0:A.shape[1],:] = A
        C = 255 - B
        image[i] = C
        
        lst_intensities = []
        g_intensities = []
        r_intensities = []
        b_intensities = []
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        a = np.mean(gray)
        pts = np.where(gray < a)
        lst_intensities.append(gray[pts[0], pts[1]])
        intensity = statistics.mean(lst_intensities[0].tolist())
        median = statistics.median(lst_intensities[0].tolist())
        
        intensity_list = intensity_list + [intensity]
        median_list = median_list + [median]

In [7]:
A.shape

(157, 309, 3)

In [8]:
sketch = sketch[sketch['Image_ID'].isin(lst)]
sketch = sketch.reset_index(drop=True)
sketch['intensity'] = intensity_list
sketch['median'] = median_list
len(sketch)

1086

In [9]:
sketch['Image_ID'].tolist() == lst

True

In [10]:
A.shape

(157, 309, 3)

In [21]:
from sklearn.model_selection import train_test_split

sketch = pd.read_csv('data/sketch_drawing.csv')

# var = 'Unique_expert'
# var = 'Drawing_expert'
# var = 'Useful_expert'
# var = 'Elegence_expert'
var = 'Creativity_expert'

mapping1 = {'Creativity_expert': 'Creativity',
                       'Useful_expert': 'Useful',
                       'Unique_expert': 'Unique',
                       'Drawing_expert': 'Drawing',
                       'Elegence_expert': 'Elegence',}
mapping = {'Creativity_expert': 'Creativity_expert_round',
                       'Useful_expert': 'Useful_expert_round',
                       'Unique_expert': 'Unique_expert_round',
                       'Drawing_expert': 'Drawing_expert_round',
                       'Elegence_expert': 'Elegence_expert_round',}

images = [image[i] for i in image.keys()]

In [22]:
sketch1 = pd.read_csv('data/sketch_drawing.csv')
text_emb = np.loadtxt('text emb 1-'+var) 

images = [image[i] for i in image.keys()]
train_df = sketch1[sketch1['set2_'+var] == 'train'] 
val_df = sketch1[sketch1['set2_'+var] == 'val'] 
test_df = sketch1[sketch1['set2_'+var] == 'test'] 

train_emb = np.array([text_emb[i] for i in train_df.index])
val_emb = np.array([text_emb[i] for i in val_df.index])
test_emb = np.array([text_emb[i] for i in test_df.index])

train_graph = [images[i] for i in train_df.index]
val_graph = [images[i] for i in val_df.index]
test_graph = [images[i] for i in test_df.index]
all_graph = [images[i] for i in range(len(images))]

In [23]:
train_cost = train_df[var]
val_cost = val_df[var]
test_cost = test_df[var]

image_shape = train_graph[0].shape

In [24]:
x_train = np.array(train_graph)/255
x_val = np.array(val_graph)/255
x_test = np.array(test_graph)/255
x_all = np.array(all_graph)/255

In [25]:
import os
import shutil
import logging
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import triang

def get_bin_idx(x):
    return min(int(x * np.float32(10)), 99)

def get_lds_kernel_window(kernel, ks, sigma):
    assert kernel in ['gaussian', 'triang', 'laplace']
    half_ks = (ks - 1) // 2
    if kernel == 'gaussian':
        base_kernel = [0.] * half_ks + [1.] + [0.] * half_ks
        kernel_window = gaussian_filter1d(base_kernel, sigma=sigma) / max(gaussian_filter1d(base_kernel, sigma=sigma))
    elif kernel == 'triang':
        kernel_window = triang(ks)
    else:
        laplace = lambda x: np.exp(-abs(x) / sigma) / (2. * sigma)
        kernel_window = list(map(laplace, np.arange(-half_ks, half_ks + 1))) / max(map(laplace, np.arange(-half_ks, half_ks + 1)))

    return kernel_window

In [26]:
#https://github.com/YyzHarry/imbalanced-regression

from collections import Counter
from scipy.ndimage import convolve1d

for i in [0]:
    # preds, labels: [Ns,], "Ns" is the number of total samples
#     preds, labels = train_df, train_cost1[var]
    labels = train_cost
    # assign each label to its corresponding bin (start from 0)
    # with your defined get_bin_idx(), return bin_index_per_label: [Ns,] 
    bin_index_per_label = [get_bin_idx(label) for label in labels]

    # calculate empirical (original) label distribution: [Nb,]
    # "Nb" is the number of bins
    Nb = max(bin_index_per_label) + 1
    num_samples_of_bins = dict(Counter(bin_index_per_label))
    emp_label_dist = [num_samples_of_bins.get(i, 0) for i in range(Nb)]

    # lds_kernel_window: [ks,], here for example, we use gaussian, ks=5, sigma=2
    lds_kernel_window = get_lds_kernel_window(kernel='gaussian', ks=5, sigma=2)
    # calculate effective label distribution: [Nb,]
    eff_label_dist = convolve1d(np.array(emp_label_dist), weights=lds_kernel_window, mode='constant')

    # Use re-weighting based on effective label distribution, sample-wise weights: [Ns,]
    eff_num_per_label = [eff_label_dist[bin_idx] for bin_idx in bin_index_per_label]
    weights = [np.float32(1 / x) for x in eff_num_per_label]
    # # calculate loss
    # loss = weighted_mse_loss(preds, labels, weights=weights)                  

In [27]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model, Input, layers, regularizers
from keras.models import load_model
from tensorflow.keras import activations
from official.nlp import optimization 

epochs = 1000
steps_per_epoch = 56
num_train_steps = steps_per_epoch * epochs
##unique: 0, drawing: 0.05, elegance: 0.05, creativity: 0.05, useful: 0.1

a = 0.00
# if var == 'Unique_expert':
#     a = 0.005
# elif var == 'Creativity_expert':
#     a = 0.02
# elif var == 'Useful_expert':
#     a = 0.1
num_warmup_steps = int(a*num_train_steps)
init_lr = 2e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

model_name1 = 'model weight/'+var+'/both_ratio-'+str(1)+'-'+var+'-best.h5'
base_model1 = load_model(model_name1)
for layer in base_model1.layers:
    layer._name = layer._name + "_a"
x = base_model1.layers[-2].output
x = layers.Dense(64)(x)
x = layers.Activation('relu')(x)
initializer = tf.keras.initializers.Constant(1/64)
out1x = layers.Dense(1, activation='relu', name='out1x', kernel_initializer=initializer)(x)
base_model = Model(base_model1.input, out1x)
# weight = np.repeat(np.array(base_model1.layers[-1].weights[0]),64).reshape(1048,64)
# base_model.layers[-3].set_weights([weight,np.zeros(64)])

merge = []
# # train simple joint model
input_tensor1 = Input(shape=image_shape)
input_tensor2 = Input(shape=512, name='input2')
model_name = 'model weight/'+var+'/new sketch-'+var+'-best.h5'
base_model.load_weights(model_name)
for i in [2,3,4]:
    base_model.layers[i]._name = base_model.layers[i].name + "_a"

'''
this can't be "base_model.layers[-5].output", which is the inceptionV3 layer.
'''
out1x = base_model.layers[-6].output
'''
tanh is better than relu
'''
out1x = layers.Activation('tanh')(out1x)
out1x = layers.Reshape((-1,2048))(out1x)
# print (out1x.shape)

drop_rate = 0.1
drop_rate1 = 0.3
final_hid = 64
head = 1
att_layers = 1
att_hid = 64
size_pool = 3

text_append = input_tensor2
text_append = layers.Dropout(drop_rate1)(text_append)

img = out1x
sketch_append =  base_model.layers[-4].output
sketch_append1 =  base_model.layers[-3].output
sketch_append1= layers.Activation('tanh')(sketch_append1)

##With attention - text on image
inpAttImg_key = img
inpAttImg_query = text_append
for layer in range(1):
    att_img = []
    concat_key = []
    for _ in range(head):
        img_key = layers.Dense(att_hid/head, use_bias=False)(inpAttImg_key) #change to tanh?
        # print (img_key.shape)
        img_key1 = layers.BatchNormalization()(img_key)
        img_key1 = layers.Activation('relu')(img_key1)
        # img_key = layers.BatchNormalization()(img_key)
        # img_key = layers.Activation('tanh')(img_key)
        img_key = layers.Dropout(drop_rate)(img_key)

        text_query = layers.Dense(att_hid/head, use_bias=False)(inpAttImg_query)
        text_query1 = layers.BatchNormalization()(text_query)
        text_query1 = layers.Activation('relu')(text_query1) 
        # text_query = layers.BatchNormalization()(text_query)
        # text_query = layers.Activation('relu')(text_query)
        text_query = layers.Dropout(drop_rate)(text_query)

        img_value = layers.Dense(att_hid/head, use_bias=False)(inpAttImg_key)
        # img_value = layers.BatchNormalization()(img_value)
        # img_value = layers.Activation('tanh')(img_value)
        img_value = layers.Dropout(drop_rate)(img_value)

        attention = layers.Dot(axes=(1,2))([text_query, img_key])
        attention = layers.Lambda(lambda x: x[0]/x[1])([attention,np.sqrt(att_hid/head)])
        attention = layers.Activation("softmax")(attention)
        # print (attention.shape)
        # print (img_value.shape)
        head_att_img = layers.Dot(axes=(1,1))([attention, img_value])
        att_img = head_att_img
        concat_key = img_key

    inpAttImg_query = att_img
    inpAttImg_key = concat_key

# img_key1 = layers.Flatten()(img_key1)
# img_key1 = layers.Dense(att_hid, use_bias=False)(img_key1)
# img_key1 = layers.BatchNormalization()(img_key1)
# img_key1 = layers.Activation('relu')(img_key1)
# img_key1 = layers.Reshape((-1,64*att_hid))(img_key1)
# merge.append(img_key1)

merge.append(att_img)
# print (att_img.shape)
merge.append(text_query1)
# print (att_img.shape)
merge.append(sketch_append1)

l_merge = layers.Concatenate(axis=1)(merge)
l_merge = layers.Dropout(drop_rate1)(l_merge) 

out5 = layers.Dense(1, activation='relu', name='out5')(l_merge)
model = Model(inputs=[base_model.input,input_tensor2], outputs=out5)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics= tf.keras.metrics.RootMeanSquaredError())

In [28]:
# model.layers

In [29]:
# base_model1.summary()
# model.summary()

In [30]:
##definition of callbacks
batch_size = 24

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=50,
#     patience=5,
    monitor="val_loss",
    restore_best_weights=True,
    verbose=1)

In [21]:
# init_lr = 3e-5
# num_warmup_steps = 0
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

# model.compile(optimizer=optimizer,
#               loss=tf.keras.losses.MeanSquaredError(),
#               metrics= tf.keras.metrics.RootMeanSquaredError())

# early_stopping_callback = tf.keras.callbacks.EarlyStopping(
#     patience=10,
#     monitor="val_loss",
#     restore_best_weights=True,
#     verbose=1)

In [22]:
tf.config.run_functions_eagerly(True)

training_history = model.fit(
    x = [x_train,train_emb],
#     x = x_train,
    y = {'out5':np.array(train_cost)},
    sample_weight={'out5': np.array(weights)},
    epochs=epochs,
    batch_size = batch_size,
    steps_per_epoch=steps_per_epoch,
    validation_data=(
        [x_val,val_emb],
#         x_val,
        {'out5':np.array(val_cost)}),
    callbacks=[early_stopping_callback,]
)

In [ ]:
idx = 0
output = pd.DataFrame()

# test_results0 = model.predict(x_test)
test_results0 = model.predict([x_test,test_emb])
test_result= test_results0.reshape(len(x_test)).tolist()
test_cost = test_df[var]

difference_array = np.subtract(test_result, test_cost)
squared_array = np.square(difference_array)
mse = squared_array.mean()

correlation_matrix = np.corrcoef(test_cost, test_result)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

output.loc[idx,'warm up'] = a
output.loc[idx,'var'] = var
output.loc[idx,'lr'] = init_lr
output.loc[idx,'mse'] = mse
output.loc[idx,'r^2'] = r_squared

In [ ]:
output

In [31]:
##save and load models
##https://stackoverflow.com/questions/46610732/how-to-freeze-some-layers-when-fine-tune-resnet50

# a = 1

# var = 'Unique_expert'
# var = 'Drawing_expert'
# var = 'Useful_expert'
# var = 'Elegence_expert'
# var = 'Creativity_expert'

# model_name = 'model weight/'+var+'/new sketch-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/new sketch-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/simple_joint_ratio-'+str(a)+'-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/new joint_'+var+'-best.h5'
# model_name = 'model weight/'+var+'/new text-'+var+'-best.h5'
model_name = 'model weight/'+var+'/attend-sketch_'+var+'-best.h5'
# model.save(model_name, save_format='h5')
##for joint model
# model.save_weights(model_name)

# from keras.models import load_model
# model = load_model(model_name)
##for text model
# model = load_model(model_name, custom_objects={'KerasLayer':hub.KerasLayer,'AdamWeightDecay': optimizer})
##for joint model
model.load_weights(model_name)

In [26]:
# maxv = 0.4064664659209161
# idx = 0
# output = pd.DataFrame()
# for lr in [4,3,2,1]:
#     for i in range(50):
#         model_name = 'model weight/'+var+'/attend-sketch_'+var+'-best.h5'
#         model.load_weights(model_name)
#         init_lr = lr*1e-5
#         num_warmup_steps = 0
#         optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                                   num_train_steps=num_train_steps,
#                                                   num_warmup_steps=num_warmup_steps,
#                                                   optimizer_type='adamw')

#         model.compile(optimizer=optimizer,
#                       loss=tf.keras.losses.MeanSquaredError(),
#                       metrics= tf.keras.metrics.RootMeanSquaredError())

#         early_stopping_callback = tf.keras.callbacks.EarlyStopping(
#             patience=3,
#             monitor="val_loss",
#             restore_best_weights=True,
#             verbose=1)

#         training_history = model.fit(
#             x = [x_train,train_emb],
#     #         x = x_train,
#             y = {'out5':np.array(train_cost)},
#             sample_weight={'out5': np.array(weights)},
#             epochs=epochs,
#             batch_size = batch_size,
#             steps_per_epoch=steps_per_epoch,
#             validation_data=(
#                 [x_val,val_emb],
#     #             x_val,
#                 {'out5':np.array(val_cost)}),
#             callbacks=[early_stopping_callback,]
#         )

#     #     test_results0 = model.predict(x_test)
#         test_results0 = model.predict([x_test,test_emb])
#         test_result= test_results0.reshape(len(x_test)).tolist()
#         test_cost = test_df[var]

#         difference_array = np.subtract(test_result, test_cost)
#         squared_array = np.square(difference_array)
#         mse = squared_array.mean()

#         correlation_matrix = np.corrcoef(test_cost, test_result)
#         correlation_xy = correlation_matrix[0,1]
#         r_squared = correlation_xy**2

#         output.loc[idx,'warm up'] = a
#         output.loc[idx,'var'] = var
#         output.loc[idx,'lr'] = init_lr
#         output.loc[idx,'mse'] = mse
#         output.loc[idx,'r^2'] = r_squared
#         idx += 1
#         print (idx)
#         if r_squared > maxv:
#             model_name1 = 'model weight/'+var+'/attend-sketch_'+var+'-best.h5'
#             model.save_weights(model_name1)
#             maxv = r_squared
#             print (maxv)

Epoch 1/1000
56/56 [==============================] - 29s 305ms/step - loss: 0.0019 - root_mean_squared_error: 0.3497 - val_loss: 0.7806 - val_root_mean_squared_error: 0.8835
Epoch 2/1000
56/56 [==============================] - 14s 245ms/step - loss: 0.0022 - root_mean_squared_error: 0.3607 - val_loss: 1.0406 - val_root_mean_squared_error: 1.0201
Epoch 3/1000
56/56 [==============================] - 13s 235ms/step - loss: 0.0020 - root_mean_squared_error: 0.3846 - val_loss: 0.7978 - val_root_mean_squared_error: 0.8932
Epoch 4/1000
56/56 [==============================] - 14s 247ms/step - loss: 0.0017 - root_mean_squared_error: 0.3637 - val_loss: 0.7491 - val_root_mean_squared_error: 0.8655
Epoch 5/1000
56/56 [==============================] - 13s 240ms/step - loss: 0.0020 - root_mean_squared_error: 0.3736 - val_loss: 0.7116 - val_root_mean_squared_error: 0.8436
Epoch 6/1000
56/56 [==============================] - 13s 239ms/step - loss: 0.0017 - root_mean_squared_error: 0.3559 - val_l

56/56 [==============================] - 14s 248ms/step - loss: 0.0019 - root_mean_squared_error: 0.3483 - val_loss: 0.8850 - val_root_mean_squared_error: 0.9407
Epoch 00008: early stopping
13
Epoch 1/1000
56/56 [==============================] - 23s 283ms/step - loss: 0.0017 - root_mean_squared_error: 0.3534 - val_loss: 0.8108 - val_root_mean_squared_error: 0.9004
Epoch 2/1000
56/56 [==============================] - 14s 255ms/step - loss: 0.0017 - root_mean_squared_error: 0.3565 - val_loss: 0.7748 - val_root_mean_squared_error: 0.8802
Epoch 3/1000
56/56 [==============================] - 13s 240ms/step - loss: 0.0019 - root_mean_squared_error: 0.3635 - val_loss: 0.8185 - val_root_mean_squared_error: 0.9047
Epoch 4/1000
56/56 [==============================] - 14s 242ms/step - loss: 0.0021 - root_mean_squared_error: 0.3743 - val_loss: 0.8246 - val_root_mean_squared_error: 0.9081
Epoch 5/1000
56/56 [==============================] - 14s 247ms/step - loss: 0.0023 - root_mean_squared_err

Epoch 00004: early stopping
26
Epoch 1/1000
56/56 [==============================] - 25s 305ms/step - loss: 0.0021 - root_mean_squared_error: 0.3562 - val_loss: 0.7288 - val_root_mean_squared_error: 0.8537
Epoch 2/1000
56/56 [==============================] - 14s 255ms/step - loss: 0.0021 - root_mean_squared_error: 0.3573 - val_loss: 0.8232 - val_root_mean_squared_error: 0.9073
Epoch 3/1000
56/56 [==============================] - 14s 258ms/step - loss: 0.0017 - root_mean_squared_error: 0.3657 - val_loss: 0.7870 - val_root_mean_squared_error: 0.8871
Epoch 4/1000
56/56 [==============================] - 15s 266ms/step - loss: 0.0020 - root_mean_squared_error: 0.3603 - val_loss: 0.8049 - val_root_mean_squared_error: 0.8972
Epoch 00004: early stopping
27
Epoch 1/1000
56/56 [==============================] - 26s 316ms/step - loss: 0.0020 - root_mean_squared_error: 0.3673 - val_loss: 1.0497 - val_root_mean_squared_error: 1.0245
Epoch 2/1000
56/56 [==============================] - 16s 278ms

Epoch 00014: early stopping
39
Epoch 1/1000
56/56 [==============================] - 29s 382ms/step - loss: 0.0019 - root_mean_squared_error: 0.3579 - val_loss: 0.7661 - val_root_mean_squared_error: 0.8753
Epoch 2/1000
56/56 [==============================] - 20s 364ms/step - loss: 0.0022 - root_mean_squared_error: 0.3559 - val_loss: 0.7566 - val_root_mean_squared_error: 0.8698
Epoch 3/1000
56/56 [==============================] - 20s 352ms/step - loss: 0.0022 - root_mean_squared_error: 0.3807 - val_loss: 0.8321 - val_root_mean_squared_error: 0.9122
Epoch 4/1000
56/56 [==============================] - 20s 359ms/step - loss: 0.0019 - root_mean_squared_error: 0.3564 - val_loss: 0.8463 - val_root_mean_squared_error: 0.9200
Epoch 5/1000
56/56 [==============================] - 20s 367ms/step - loss: 0.0017 - root_mean_squared_error: 0.3480 - val_loss: 0.7790 - val_root_mean_squared_error: 0.8826
Epoch 00005: early stopping
40
Epoch 1/1000
56/56 [==============================] - 29s 390ms

56/56 [==============================] - 26s 459ms/step - loss: 0.0018 - root_mean_squared_error: 0.3548 - val_loss: 0.7920 - val_root_mean_squared_error: 0.8899
Epoch 3/1000
56/56 [==============================] - 21s 379ms/step - loss: 0.0019 - root_mean_squared_error: 0.3552 - val_loss: 0.8165 - val_root_mean_squared_error: 0.9036
Epoch 4/1000
56/56 [==============================] - 22s 393ms/step - loss: 0.0018 - root_mean_squared_error: 0.3531 - val_loss: 0.7676 - val_root_mean_squared_error: 0.8762
Epoch 5/1000
56/56 [==============================] - 24s 424ms/step - loss: 0.0021 - root_mean_squared_error: 0.3516 - val_loss: 0.7949 - val_root_mean_squared_error: 0.8916
Epoch 6/1000
56/56 [==============================] - 24s 438ms/step - loss: 0.0020 - root_mean_squared_error: 0.3617 - val_loss: 0.7535 - val_root_mean_squared_error: 0.8680
Epoch 7/1000
56/56 [==============================] - 23s 417ms/step - loss: 0.0016 - root_mean_squared_error: 0.3361 - val_loss: 0.7830 -

56/56 [==============================] - 38s 543ms/step - loss: 0.0018 - root_mean_squared_error: 0.3672 - val_loss: 0.8648 - val_root_mean_squared_error: 0.9299
Epoch 2/1000
56/56 [==============================] - 28s 502ms/step - loss: 0.0020 - root_mean_squared_error: 0.3510 - val_loss: 0.7532 - val_root_mean_squared_error: 0.8679
Epoch 3/1000
56/56 [==============================] - 27s 480ms/step - loss: 0.0019 - root_mean_squared_error: 0.3586 - val_loss: 0.7632 - val_root_mean_squared_error: 0.8736
Epoch 4/1000
56/56 [==============================] - 28s 502ms/step - loss: 0.0019 - root_mean_squared_error: 0.3575 - val_loss: 0.7702 - val_root_mean_squared_error: 0.8776
Epoch 5/1000
56/56 [==============================] - 28s 506ms/step - loss: 0.0015 - root_mean_squared_error: 0.3542 - val_loss: 0.9457 - val_root_mean_squared_error: 0.9725
Epoch 00005: early stopping
64
Epoch 1/1000
56/56 [==============================] - 37s 533ms/step - loss: 0.0020 - root_mean_squared_err

ResourceExhaustedError:  OOM when allocating tensor with shape[192,192,1,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/inception_v3_a/conv2d_66/Conv2D
 (defined at C:\Users\sbyli\anaconda3\envs\tf-transf\lib\site-packages\keras\layers\convolutional.py:231)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_1603595]

Errors may have originated from an input operation.
Input Source operations connected to node model_1/inception_v3_a/conv2d_66/Conv2D:
In[0] model_1/inception_v3_a/activation_65/Relu (defined at C:\Users\sbyli\anaconda3\envs\tf-transf\lib\site-packages\keras\backend.py:4867)	
In[1] model_1/inception_v3_a/conv2d_66/Conv2D/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "C:\Users\sbyli\anaconda3\envs\tf-trans